In [1]:
# Imports and data loading
!pip install -U imbalanced-learn==0.10.1 scikit-learn==1.2.2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_recall_curve, classification_report, fbeta_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Load datasets
df_train = pd.read_csv('/kaggle/input/creditcsv/cs-training.csv', index_col=0)
df_test = pd.read_csv('/kaggle/input/creditcsv/cs-test.csv', index_col=0)
df_submission = pd.read_csv('/kaggle/input/creditcsv/sampleEntry.csv')

# Basic info
print(df_train.info())
print(df_train.head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 4.8 MB/s eta 0:00:0000:01
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
<class 'pandas.core.frame.DataFrame'>
Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7

In [5]:
# Advanced data preprocessing and feature engineering

# Impute missing with KNN
imputer = KNNImputer(n_neighbors=5)
df_train_imputed = df_train.copy()
df_train_imputed[['MonthlyIncome', 'NumberOfDependents']] = imputer.fit_transform(df_train_imputed[['MonthlyIncome', 'NumberOfDependents']])

df_test_imputed = df_test.copy()
df_test_imputed[['MonthlyIncome', 'NumberOfDependents']] = imputer.transform(df_test_imputed[['MonthlyIncome', 'NumberOfDependents']])

# Feature engineering example: Debt to income ratio squared, Age buckets
df_train_imputed['DebtRatio_sq'] = df_train_imputed['DebtRatio'] ** 2
df_train_imputed['Age_bucket'] = pd.cut(df_train_imputed['age'], bins=[0,25,40,60,100], labels=[0,1,2,3])

df_test_imputed['DebtRatio_sq'] = df_test_imputed['DebtRatio'] ** 2
df_test_imputed['Age_bucket'] = pd.cut(df_test_imputed['age'], bins=[0,25,40,60,100], labels=[0,1,2,3])

In [2]:
# Prepare data for modeling
target = 'SeriousDlqin2yrs'
features = [c for c in df_train.columns if c != target]

# Si hay categóricas, hacer dummy:
X = pd.get_dummies(df_train[features])

y = df_train[target]

# Imputar faltantes con mediana (en columnas numéricas)
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42, stratify=y
)

# Escalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Balance con SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)

In [5]:
# Define custom F2 scorer and threshold finder

def f2_scorer(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2)

def find_best_threshold_f2(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    f2_scores = 5 * (precision * recall) / (4 * precision + recall + 1e-9)
    best_idx = np.argmax(f2_scores)
    return thresholds[best_idx], f2_scores[best_idx]


In [6]:
# Hyperparameter tuning with Optuna for LightGBM

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = LGBMClassifier(**param)
    model.fit(X_train_bal, y_train_bal)
    y_val_prob = model.predict_proba(X_val_scaled)[:, 1]
    
    best_thresh, best_f2 = find_best_threshold_f2(y_val, y_val_prob)
    return best_f2

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best params:", study.best_params)
print("Best F2 score:", study.best_value)


[I 2025-06-30 20:18:33,315] A new study created in memory with name: no-name-f2798fc3-fc6a-4b92-9df3-f6d3627a3d47


[LightGBM] [Info] Number of positive: 111979, number of negative: 111979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2546
[LightGBM] [Info] Number of data points in the train set: 223958, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2025-06-30 20:18:37,589] Trial 0 finished with value: 0.49597469779716935 and parameters: {'n_estimators': 392, 'learning_rate': 0.029657457039949927, 'num_leaves': 20, 'max_depth': 5, 'min_child_samples': 8, 'subsample': 0.7210440763364347, 'colsample_bytree': 0.8188106946965754, 'reg_alpha': 0.7230596559550575, 'reg_lambda': 0.2286419615690013}. Best is trial 0 with value: 0.49597469779716935.


[LightGBM] [Info] Number of positive: 111979, number of negative: 111979
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013909 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2546
[LightGBM] [Info] Number of data points in the train set: 223958, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[I 2025-06-30 20:18:51,867] Trial 1 finished with value: 0.5043992715468248 and parameters: {'n_estimators': 1336, 'learning_rate': 0.03977385813151292, 'num_leaves': 33, 'max_depth': 5, 'min_child_samples': 35, 'subsample': 0.6833788252054572, 'colsample_bytree': 0.8409103436138448, 'reg_alpha': 0.5507491022269766, 'reg_lambda': 0.5434834964078473}. Best is trial 1 with value: 0.5043992715468248.


...

[LightGBM] [Info] Number of positive: 111979, number of negative: 111979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2546
[LightGBM] [Info] Number of data points in the train set: 223958, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[I 2025-06-30 20:27:20,141] Trial 48 finished with value: 0.5107023680734549 and parameters: {'n_estimators': 241, 'learning_rate': 0.022861844105512785, 'num_leaves': 123, 'max_depth': 3, 'min_child_samples': 25, 'subsample': 0.5908731536375549, 'colsample_bytree': 0.5619581334884983, 'reg_alpha': 0.34322438021503365, 'reg_lambda': 0.13660756722650136}. Best is trial 43 with value: 0.5124010952595783.


[LightGBM] [Info] Number of positive: 111979, number of negative: 111979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2546
[LightGBM] [Info] Number of data points in the train set: 223958, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

[I 2025-06-30 20:27:24,031] Trial 49 finished with value: 0.5091781549317455 and parameters: {'n_estimators': 491, 'learning_rate': 0.015668100974729877, 'num_leaves': 121, 'max_depth': 3, 'min_child_samples': 25, 'subsample': 0.5545194704004348, 'colsample_bytree': 0.5224406307260109, 'reg_alpha': 0.24085165588195467, 'reg_lambda': 0.12414953616348667}. Best is trial 43 with value: 0.5124010952595783.


Best params: {'n_estimators': 299, 'learning_rate': 0.014847005317051118, 'num_leaves': 87, 'max_depth': 4, 'min_child_samples': 17, 'subsample': 0.6028413703548049, 'colsample_bytree': 0.574634376254013, 'reg_alpha': 0.26274869957642943, 'reg_lambda': 0.012416079117458106}
Best F2 score: 0.5124010952595783


In [7]:
# Train final LightGBM with best params

best_params = study.best_params
best_params['random_state'] = 42
best_params['n_jobs'] = -1

final_lgbm = LGBMClassifier(**best_params)
final_lgbm.fit(X_train_bal, y_train_bal)

# Validation predictions
y_val_prob_final = final_lgbm.predict_proba(X_val_scaled)[:,1]
best_thresh, best_f2 = find_best_threshold_f2(y_val, y_val_prob_final)
print(f"Best threshold: {best_thresh:.4f}, Best F2-score: {best_f2:.4f}")

y_val_pred_final = (y_val_prob_final >= best_thresh).astype(int)
print(classification_report(y_val, y_val_pred_final))


[LightGBM] [Info] Number of positive: 111979, number of negative: 111979
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2546
[LightGBM] [Info] Number of data points in the train set: 223958, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [10]:
# Prepare test set and make predictions
df_test_imputed = df_test.copy()
imputer = SimpleImputer(strategy='median')
df_test_imputed[['MonthlyIncome', 'NumberOfDependents']] = imputer.fit_transform(
    df_test_imputed[['MonthlyIncome', 'NumberOfDependents']]
)

# Preprocess test set same as train
X_test = df_test_imputed[features]
X_test_scaled = scaler.transform(X_test)

# Predict probabilities
y_test_prob = final_lgbm.predict_proba(X_test_scaled)[:, 1]

# Predict labels based on best threshold
y_test_pred = (y_test_prob >= best_thresh).astype(int)

# Prepare submission dataframe with only Id and prediction probability
submission = pd.DataFrame({
    'Id': df_submission['Id'],
    'Probability': y_test_prob
})

submission.to_csv('submission.csv', index=False)
print("Submission saved!")

Submission saved!
